In [8]:
import glob
import cv2
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=np.inf)
import os

In [58]:
images = glob.glob("./font/*/*/*.png"); # array of image file paths (features)

for file in images:
    # loading image as a grayscale
    image = cv2.imread(file,  cv2.IMREAD_GRAYSCALE);
    normalized_image = image / 255.0 # normalizing

    # handwriting extraction and obtaining a binary mask
    _, binary_mask = cv2.threshold(normalized_image, 0.7, 1, cv2.THRESH_BINARY_INV)
    binary_mask_path = f'{file[0:-4]}_binary_mask.png';
    #print(binary_mask_path)
    if not cv2.imwrite(binary_mask_path, (binary_mask * 255).astype(np.uint8)):
        raise Exception("Could not write image")


./font/numbers/number_zero/number_zero_3_binary_mask.png
./font/numbers/number_zero/number_zero_2_binary_mask.png
./font/numbers/number_zero/number_zero_19_binary_mask.png
./font/numbers/number_zero/number_zero_1_binary_mask.png
./font/numbers/number_zero/number_zero_18_binary_mask.png
./font/numbers/number_zero/number_zero_20_binary_mask.png
./font/numbers/number_zero/number_zero_5_binary_mask.png
./font/numbers/number_zero/number_zero_4_binary_mask.png
./font/numbers/number_zero/number_zero_6_binary_mask.png
./font/numbers/number_zero/number_zero_7_binary_mask.png
./font/numbers/number_zero/number_zero_13_binary_mask.png
./font/numbers/number_zero/number_zero_12_binary_mask.png
./font/numbers/number_zero/number_zero_10_binary_mask.png
./font/numbers/number_zero/number_zero_9_binary_mask.png
./font/numbers/number_zero/number_zero_8_binary_mask.png
./font/numbers/number_zero/number_zero_11_binary_mask.png
./font/numbers/number_zero/number_zero_15_binary_mask.png
./font/numbers/number_z

In [ ]:
for binary_mask in binary_masks:
    